# This code generate a Flask Web APP to predict and classify images, detecting if there's Violence or not

This codes includes function to create thumbnails of the videos in the dataset.

It uses the models generated on the Train_violence_model.ipynb file

In [1]:
# Install necesary libs
!pip install Flask
!pip install imgaug
!pip install pytube

In [ ]:
# Used once for creating thumbnails of videos
import os
import subprocess

# Specify the directory containing your videos
video_directory = './archive/Real Life Violence Dataset/NonViolence'

# Specify the directory where you want to save the thumbnails
thumbnail_directory = './archive/Real Life Violence Dataset/NonViolence/thumbnails'

# Ensure the thumbnail directory exists
if not os.path.exists(thumbnail_directory):
    os.makedirs(thumbnail_directory)

# Iterate over each file in the video directory
for filename in os.listdir(video_directory):
    # Check if the file is a video (you might need to adjust the condition based on your video formats)
    if filename.endswith(('.mp4', '.avi', '.mkv')):
        # Construct the full path to the video file
        video_path = os.path.join(video_directory, filename)
        
        # Construct the output path for the thumbnail
        thumbnail_path = os.path.join(thumbnail_directory, filename + '.jpg')
        
        # Use FFmpeg to generate a thumbnail from the video
        # This command extracts a frame at 10 seconds into the video and saves it as a JPEG
        subprocess.call(['ffmpeg', '-ss', '00:00:1.000', '-i', video_path, '-vframes', '1', thumbnail_path])

print("Thumbnails generated successfully.")

In [2]:
# Function to Extract frames from videos
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

IMG_SIZE = 128
ColorChannels = 3

def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)

    import math
    rate = math.floor(vidcap.get(3))
    count = 0

    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()

        if success:
            # skipping frames to avoid duplications
            if (ID % 7 == 0):
                flip = iaa.Fliplr(1.0)
                zoom = iaa.Affine(scale=1.3)
                random_brightness = iaa.Multiply((1, 1.3))
                rotate = iaa.Affine(rotate=(-25, 25))

                image_aug = flip(image = image)
                image_aug = random_brightness(image = image_aug)
                image_aug = zoom(image = image_aug)
                image_aug = rotate(image = image_aug)

                rgb_img = cv2.cvtColor(image_aug, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)

            count += 1
        else:
            break

    vidcap.release()

    return ImageFrames

In [3]:
#Load trained model
from keras.models import load_model

model = load_model('model.h5')

In [4]:
def avg_max(video_name,confidence):
    filen = video_name.split("\\",2)[1]
    v_nv = filen.split("_")[0]
    if v_nv == "NV":
        return np.average(confidence)
    elif v_nv == "V":
        return max(confidence)

def with_moviepy(filename):
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(filename)
    duration       = clip.duration
    return duration

In [ ]:
from flask import Flask, render_template, request, Response, redirect, url_for
import os
import random
from PIL import Image
from keras.preprocessing import image
import numpy as np
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from pytube import YouTube
from pytube.innertube import _default_clients

_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]


thumbnails = os.path.join('static','thumbnails')
videos = os.path.join('static')
template_dir = os.path.abspath('./templates')
app = Flask(__name__, template_folder=template_dir)
app.config['UPLOAD_FOLDER'] = thumbnails

@app.route('/')
def list_videos():
    #List random file from directory
    video_directory = './static/'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    all_frames = len(video_to_frames(full_filename_video))
    result = []
    for i in range(all_frames):
        
        image_file = video_to_frames(full_filename_video)[i]
        im = Image.fromarray(image_file)
        im.save("inference.jpeg")
        
        img_width, img_height = 128, 128
        img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        result.append(model.predict(img))
        
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = avg_max(full_filename_video,result))

@app.route('/select_video', methods=['POST'])
def select_video():
    #List random file from directory
    video_directory = './static'
    all_files = os.listdir(video_directory)
    video_files = [f for f in all_files if f.endswith(('.mp4', '.avi', '.mkv'))]
    random_videos = random.sample(video_files, min(1, len(video_files)))
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], random_videos[0] + '.jpg')
    full_filename_video = os.path.join(videos, random_videos[0])

    all_frames = len(video_to_frames(full_filename_video))
    result = []
    for i in range(all_frames):
        
        image_file = video_to_frames(full_filename_video)[i]
        im = Image.fromarray(image_file)
        im.save("inference.jpeg")
        
        img_width, img_height = 128, 128
        img = image.load_img("inference.jpeg", target_size = (img_width, img_height))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        result.append(model.predict(img))
        
    return render_template('videos.html', videos=random_videos, images = full_filename, detection = avg_max(full_filename_video,result))

@app.route('/Youtube_form', methods=['GET', 'POST'])
def Youtube_form():
    if request.method == 'POST':
        yt = YouTube(request.form['link'], use_oauth=True, allow_oauth_cache=True)
        new_video = yt.streams.first().download('')
        ytvideo = new_video.split("\\",5)[5]
        
        if with_moviepy(ytvideo) <= 5.0:
            all_frames = len(video_to_frames(ytvideo))
            result = []
            for i in range(all_frames):
                
                image_file = video_to_frames(ytvideo)[i]
                im = Image.fromarray(image_file)
                im.save("./static/inference.jpeg")
                
                img_width, img_height = 128, 128
                img = image.load_img("./static/inference.jpeg", target_size = (img_width, img_height))
                img = image.img_to_array(img)
                img = np.expand_dims(img, axis = 0)
                result.append(model.predict(img))
        elif with_moviepy(ytvideo) > 5.0:
            ffmpeg_extract_subclip(ytvideo, 5, 10, targetname="video.mp4")
            all_frames = len(video_to_frames("video.mp4"))
            result = []
            for i in range(all_frames):
                
                image_file = video_to_frames("video.mp4")[i]
                im = Image.fromarray(image_file)
                im.save("./static/inference.jpeg")
                
                img_width, img_height = 128, 128
                img = image.load_img("./static/inference.jpeg", target_size = (img_width, img_height))
                img = image.img_to_array(img)
                img = np.expand_dims(img, axis = 0)
                result.append(model.predict(img))
                
        return render_template('Youtube_results.html', videos=ytvideo, images = "./inference.jpeg", detection = np.average(result))
     
    return render_template('Youtube_form.html')


if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000
Press CTRL+C to quit


1/1 [==============================] - 0s 30ms/step


127.0.0.1 - - [22/Apr/2024 21:26:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 21:26:46] "GET /static/thumbnails/V_800.mp4.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 21:26:46] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 27ms/step


127.0.0.1 - - [22/Apr/2024 21:26:50] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 21:26:50] "GET /static/thumbnails/V_169.mp4.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 27ms/step


127.0.0.1 - - [22/Apr/2024 21:26:54] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 21:26:54] "GET /static/thumbnails/NV_238.mp4.jpg HTTP/1.1" 200 -


1/1 [==============================] - 0s 27ms/step


127.0.0.1 - - [22/Apr/2024 21:27:07] "POST /select_video HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 21:27:07] "GET /static/thumbnails/V_93.mp4.jpg HTTP/1.1" 200 -
